In [1]:
import pandas as pd
import datetime as dt

In [2]:
f = lambda s: dt.datetime.strptime(s,'%d-%m-%Y %H:%M:%S')
# df_bn5 = pd.read_csv("data/historical/BankNiftyFut-Sep-5min-20200908.csv", sep="\t", parse_dates=['Date Time'], date_parser=f, encoding="utf_16_LE") 
df_bn5 = pd.read_csv("data/historical/NiftyFut-Sep-5min-20200908.csv", sep="\t", parse_dates=['Date Time'], encoding="utf_16_LE") 

#### Remove last empty column and set date time as index

In [3]:
df_bn5 = df_bn5[df_bn5.columns[:-1]]
df_bn5.set_index('Date Time', inplace=True)

#### Apply rolling mean columns for data

In [4]:
df_bn5['SMA6'] = df_bn5['Close'].rolling(6).mean()
df_bn5['SMA21'] = df_bn5['Close'].rolling(21).mean()

## Input the evaluation date below

In [5]:
eval_date = "2020-09-08"

#### Extract a separate dataframe for the evaluation date

In [6]:
today_df_bn5 = df_bn5[eval_date]

In [7]:
today_df_bn5.head(5)

,Open,Low,High,Close,Volume,SMA6,SMA21
Date Time,,,,,,,
2020-09-08 09:15:00,11348.0,11332.05,11370.90,11348.30,412653,11376.758333,11346.145238
2020-09-08 09:20:00,11354.5,11348.95,11366.05,11359.65,254550,11375.691667,11346.759524
2020-09-08 09:25:00,11362.4,11350.50,11362.40,11354.70,119775,11371.491667,11347.400000
2020-09-08 09:30:01,11355.9,11341.25,11357.00,11344.90,158250,11364.141667,11346.442857
2020-09-08 09:35:00,11344.9,11342.00,11363.25,11360.10,146625,11359.600000,11346.507143


### Strategy function

In [10]:
def backtest_Intraday_5min_strategy(df):
    SELL = False
    BUY = False
    SELL_PRICE = 0
    BUY_PRICE = 0
    GAIN_POINTS = 0
    SELL_STOPLOSS = 0
    BUY_STOPLOSS = 0
    STOPLOSS = 25
    SMA6_dead_band = 15
    Total_Signals = 0
    Total_Sell_Signal = 0
    Total_Buy_Signal = 0
    start_time = dt.datetime.strptime("10:00", "%H:%M").time()
    close_time = dt.datetime.strptime("15:15", "%H:%M").time()
    entry_close_time = dt.datetime.strptime("15:00", "%H:%M").time()
    print("Time\t\t\t", "Close\t\t", "SMA6\t\t", "SMA21\t\t", "SIGNAL\t", "TYPE\t\t",  "Points\t",)
    
    for row in df.itertuples():
        SMA6_upper_band = row.SMA6 + SMA6_dead_band
        SMA6_lower_band = row.SMA6 - SMA6_dead_band
        
        # SELL STOPLOSS
        if row.Index.time() >= start_time and row.Index.time() <= close_time: 
            points = 0
            if SELL and row.Close > SELL_STOPLOSS:
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Buy_Signal += 1
                points = SELL_PRICE - row.Close
                GAIN_POINTS = GAIN_POINTS + points
                SELL_PRICE = 0
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "SELL STOPLOSS\t", round(points,2))
            
            # BUY STOPLOSS
            points = 0
            if BUY and row.Close < BUY_STOPLOSS:
                points = row.Close - BUY_PRICE
                GAIN_POINTS = GAIN_POINTS + points
                BUY_PRICE = 0
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Sell_Signal += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "BUY STOPLOSS\t", round(points,2))
            
            # SELL ENTRY
            points = 0
            if row.Close < SMA6_lower_band and \
                    row.Index.time() < entry_close_time and \
                    SELL is False:
                SELL_PRICE = row.Close
                SELL_STOPLOSS = SELL_PRICE + STOPLOSS
                if BUY_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                SELL = True
                BUY = False
                Total_Sell_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL ENTRY\t", round(points,2))
             
            # BUY ENTRY
            points = 0
            if row.Close > SMA6_upper_band and \
                    row.Index.time() < entry_close_time and \
                    BUY is False:
                BUY_PRICE = row.Close
                BUY_STOPLOSS = BUY_PRICE - STOPLOSS
                if SELL_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                BUY = True
                SELL = False
                Total_Buy_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t", row.Close,  "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY ENTRY\t", round(points,2))
            
            # DAY Close
            points = 0    
            if row.Index.time() >= close_time:
                if SELL:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = SELL_PRICE - row.Close
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY DAYCLOSE\t", round(points,2))

                if BUY:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = row.Close - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL DAYCLOSE\t", round(points,2))
                    
                    
    
    print("\n\nTotal Signals:\t\t", Total_Signals)
    print("Total Sell Signals:\t", Total_Buy_Signal)
    print("Total Buy Signals:\t", Total_Sell_Signal)
    print("Total Points Gained:\t", round(GAIN_POINTS,2))

In [11]:
backtest_Intraday_5min_strategy(today_df_bn5)

Time			 Close		 SMA6		 SMA21		 SIGNAL	 TYPE		 Points	
2020-09-08 10:25:00 	 11401.5 	 11385.22 	 11371.95 	 BUY	 BUY ENTRY	 0
2020-09-08 13:45:00 	 11397.2 	 11414.47 	 11422.41 	 SELL	 SELL ENTRY	 -4.3
2020-09-08 15:15:00 	 11317.6 	 11329.72 	 11375.63 	 BUY	 BUY DAYCLOSE	 79.6


Total Signals:		 3
Total Sell Signals:	 2
Total Buy Signals:	 1
Total Points Gained:	 75.3
